# (강의) 감성분석 파고들기

신록예찬  
2024-09-18

# 1. Imports

In [131]:
import datasets
import transformers
import evaluate
import numpy as np
import torch

# 2. 이전코드

`-` Step1~4를 위한 준비

In [2]:
# ## Step1 
# 데이터불러오기 = datasets.load_dataset
# 데이터전처리하기1 = 토크나이저 = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased") 
# def 데이터전처리하기2(examples):
#     return 데이터전처리하기1(examples["text"], truncation=True)
# ## Step2 
# 인공지능생성하기 = transformers.AutoModelForSequenceClassification.from_pretrained
# ## Step3 
# 데이터콜렉터 = transformers.DataCollatorWithPadding(tokenizer=토크나이저)
# def 평가하기(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     accuracy = evaluate.load("accuracy")
#     return accuracy.compute(predictions=predictions, references=labels)
# 트레이너세부지침생성기 = transformers.TrainingArguments
# 트레이너생성기 = transformers.Trainer
# ## Step4 
# 강인공지능생성하기 = transformers.pipeline

`-` Step 1~4

In [269]:
# ## Step1 
# 데이터 = 데이터불러오기('imdb')
# 전처리된데이터 = 데이터.map(데이터전처리하기2,batched=True)
# 전처리된훈련자료, 전처리된검증자료 = 전처리된데이터['train'], 전처리된데이터['test']
# ## Step2 
# 인공지능 = 인공지능생성하기("distilbert/distilbert-base-uncased", num_labels=2)
# ## Step3 
# 트레이너세부지침 = 트레이너세부지침생성기(
#     output_dir="my_awesome_model",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=2, # 전체문제세트를 2번 공부하라..
#     weight_decay=0.01,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     push_to_hub=False,
# )
# 트레이너 = 트레이너생성기(
#     model=인공지능,
#     args=트레이너세부지침,
#     train_dataset=전처리된훈련자료,
#     eval_dataset=전처리된검증자료,
#     tokenizer=토크나이저,
#     data_collator=데이터콜렉터,
#     compute_metrics=평가하기,
# )
# 트레이너.train()
# ## Step4 
# 강인공지능 = 강인공지능생성하기("sentiment-analysis", model="my_awesome_model/checkpoint-1563")
# print(강인공지능("This movie was a huge disappointment."))
# print(강인공지능("This was a masterpiece."))

# 3. DataSet

`-` 가장 중요한 원초적 질문: 데이터 셋을 바꿔치기 하려면?

`-` 내가 원하는 데이터를 아래와 같은 형태로 정리가능해야함.

In [24]:
데이터 = datasets.load_dataset('imdb')
데이터

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [25]:
데이터, type(데이터)

(DatasetDict({
     train: Dataset({
         features: ['text', 'label'],
         num_rows: 25000
     })
     test: Dataset({
         features: ['text', 'label'],
         num_rows: 25000
     })
     unsupervised: Dataset({
         features: ['text', 'label'],
         num_rows: 50000
     })
 }),
 datasets.dataset_dict.DatasetDict)

In [26]:
데이터['train'], type(데이터['train'])

(Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 }),
 datasets.arrow_dataset.Dataset)

`-` `datasets.arrow_dataset.Dataset` 의 인스턴스 만들기

아래와 같은 함수가 있음

In [27]:
# datasets.Dataset.from_dict?
# 클래스메소드

이 함수를 쓰는 방법?

### $\star$ `datasets.Dataset.from_dict` 사용법

`datasets.Dataset.from_dict`는 Python의 딕셔너리(`dict`)를 `Dataset`
객체로 변환하는 함수입니다. 주로 딕셔너리 형태의 데이터를 빠르게
데이터셋으로 변환할 때 사용됩니다.

**간단한 사용법**

``` python
from datasets import Dataset

# 딕셔너리를 Dataset으로 변환
data_dict = {
    'text': ["Hello world", "How are you?", "Fine, thanks!"],
    'label': [0, 1, 1]
}

# Dataset 생성
dataset = Dataset.from_dict(data_dict)

# 출력
print(dataset)
```

**주요 매개변수:** - `mapping`: 필수, 문자열을 키로 하고 리스트 또는
배열을 값으로 하는 딕셔너리. - `features`: 선택, 데이터셋의 각 필드
타입을 정의. - `info`: 선택, 데이터셋에 대한 추가 정보(설명, 인용 등). -
`split`: 선택, 데이터셋의 나누기(‘train’, ‘test’ 등).

**반환값:** - `Dataset`: PyArrow 기반의 데이터셋 객체.

In [28]:
train_dict = {
    'text': [
        "I prefer making decisions based on logic and objective facts.",
        "I always consider how others might feel when making a decision.",
        "Data and analysis drive most of my decisions.",
        "I rely on my empathy and personal values to guide my choices."
    ],
    'label': [0, 1, 0, 1]  # 0은 T(사고형), 1은 F(감정형)
}

test_dict = {
    'text': [
        "I find it important to weigh all the pros and cons logically.",
        "When making decisions, I prioritize harmony and people's emotions."
    ],
    'label': [0, 1]  # 0은 T(사고형), 1은 F(감정형)
}
train = datasets.Dataset.from_dict(train_dict)
test = datasets.Dataset.from_dict(test_dict)

In [29]:
train

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [30]:
test

Dataset({
    features: ['text', 'label'],
    num_rows: 2
})

`-` `datasets.dataset_dict.DatasetDict`의 인스턴스 만들기

In [31]:
_데이터 = datasets.DatasetDict({'train':train, 'test':test})

In [32]:
_데이터

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
})

`-` 일단 아래와 같은 형태로 분석할 데이터를 정리할 수만 있다면, 나머지는
코드를 복붙해서 일단 돌릴 수 있음.

``` python
train_dict = {
    'text': [
        "I prefer making decisions based on logic and objective facts.",
        "I always consider how others might feel when making a decision.",
        "Data and analysis drive most of my decisions.",
        "I rely on my empathy and personal values to guide my choices."
    ],
    'label': [0, 1, 0, 1]  # 0은 T(사고형), 1은 F(감정형)
}

test_dict = {
    'text': [
        "I find it important to weigh all the pros and cons logically.",
        "When making decisions, I prioritize harmony and people's emotions."
    ],
    'label': [0, 1]  # 0은 T(사고형), 1은 F(감정형)
}
```

# 4. 토크나이저

`-` 기본적인 사용방법

In [264]:
토크나이저 = 데이터전처리하기1 = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased") 

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(

In [38]:
#토크나이저?

> **$\star$ `토크나이저` 사용법 (ref: ChatGPT)**
>
> **주요 파라미터**:
>
> 1.  **text**:
>     -   `Union[str, List[str], List[List[str]]]`
>     -   주어진 텍스트를 토큰화합니다. 이 텍스트는 문자열일 수도 있고,
>         문자열의 리스트 또는 리스트 안의 리스트일 수도 있습니다.
> 2.  **text_pair**:
>     -   `Union[str, List[str], List[List[str]], NoneType]`
>     -   두 개의 텍스트를 함께 모델에 입력할 때 사용됩니다. 예를 들어,
>         질문-답변 쌍 같은 경우 이 두 번째 텍스트를 넣습니다.
> 3.  **text_target**:
>     -   `Union[str, List[str], List[List[str]]]`
>     -   토큰화를 할 때 목표(target) 텍스트에 해당하는 부분입니다. 주로
>         시퀀스 생성 모델에서 활용됩니다.
> 4.  **text_pair_target**:
>     -   `Union[str, List[str], List[List[str]], NoneType]`
>     -   위의 `text_pair`와 유사하게 목표(target) 텍스트의 두 번째
>         텍스트를 나타냅니다.
> 5.  **add_special_tokens**:
>     -   `bool`
>     -   문장의 시작, 끝, 구분자 같은 특별한 토큰을 추가할지 여부를
>         결정합니다. 기본값은 `True`입니다.
> 6.  **padding**:
>     -   `Union[bool, str, transformers.utils.generic.PaddingStrategy]`
>     -   문장 길이가 다를 때 패딩을 넣어 문장의 길이를 동일하게
>         맞춥니다. 패딩 전략에는 `True`, `False`, `'longest'`,
>         `'max_length'` 등이 있습니다.
> 7.  **truncation**:
>     -   `Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy]`
>     -   문장이 너무 길 경우 지정된 최대 길이에 맞춰 잘라내는
>         옵션입니다. 전략에는 `True`, `False`, `'longest_first'`,
>         `'only_first'`, `'only_second'` 등이 있습니다.
> 8.  **max_length**:
>     -   `Optional[int]`
>     -   문장의 최대 길이를 설정합니다. `None`일 경우 기본 설정을
>         따릅니다.
> 9.  **stride**:
>     -   `int`
>     -   텍스트를 자를 때 중첩을 만들기 위한 옵션입니다. 즉, 자른
>         부분과 다음 부분 사이의 겹치는 범위를 설정합니다.
> 10. **is_split_into_words**:
>     -   `bool`
>     -   텍스트가 이미 단어 단위로 분리되어 있는지 여부를 나타냅니다.
>         기본적으로는 `False`로, 텍스트가 단어 단위로 분리되지 않았다고
>         가정합니다.
> 11. **return_tensors**:
>     -   `Union[str, transformers.utils.generic.TensorType, NoneType]`
>     -   출력 형식으로 텐서를 반환할지 여부를 설정합니다.
>         `'pt'`(PyTorch), `'tf'`(TensorFlow), `'np'`(NumPy) 등을 지정할
>         수 있습니다.
> 12. **return_token_type_ids**:
>     -   `Optional[bool]`
>     -   토큰 타입 ID를 반환할지 여부를 설정합니다. 주로 두 개의 문장을
>         함께 처리할 때 문장을 구분하기 위해 사용됩니다.
> 13. **return_attention_mask**:
>     -   `Optional[bool]`
>     -   `attention_mask`를 반환할지 여부를 설정합니다. 패딩된 토큰이
>         모델의 어텐션에 영향을 주지 않도록 마스크를 설정합니다.
> 14. **return_overflowing_tokens**:
>     -   `bool`
>     -   텍스트가 최대 길이를 초과하는 경우, 잘린 토큰을 반환할지
>         여부를 결정합니다.
> 15. **return_special_tokens_mask**:
>     -   `bool`
>     -   특별한 토큰에 대한 마스크를 반환할지 여부를 설정합니다.
> 16. **return_offsets_mapping**:
>     -   `bool`
>     -   텍스트의 각 토큰이 원본 텍스트에서 어느 위치에 있는지 나타내는
>         오프셋 맵핑을 반환할지 여부를 설정합니다.
> 17. **return_length**:
>     -   `bool`
>     -   토큰화된 문장의 길이를 반환할지 여부를 설정합니다.
> 18. **verbose**:
>     -   `bool`
>     -   디버깅 메시지를 출력할지 여부를 설정합니다. 기본값은 `True`로
>         설정되어 있습니다.
>
> **사용 예시**:
>
> ``` python
> from transformers import AutoTokenizer
>
> # 토크나이저 불러오기
> tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
>
> # 텍스트 토큰화
> encoding = tokenizer(
>     text="Hello, how are you?",
>     padding=True,
>     truncation=True,
>     max_length=10,
>     return_tensors='pt'
> )
>
> print(encoding)
> ```
>
> 이 코드에서는 “Hello, how are you?”라는 텍스트를 `bert-base-uncased`
> 토크나이저로 토큰화하고, 패딩과 트렁케이션을 적용하며, PyTorch 텐서
> 형식으로 반환하도록 설정했습니다.
>
> 이러한 파라미터는 주로 자연어 처리(NLP) 모델을 훈련하거나 추론할 때
> 데이터 전처리 과정에서 많이 사용됩니다.

`-` 기본사용1: 단어별로 다른숫자를 맵핑 + 처음과 끝은 항상 `101`, `102`

In [39]:
토크나이저("hi hello")

{'input_ids': [101, 7632, 7592, 102], 'attention_mask': [1, 1, 1, 1]}

In [40]:
토크나이저("hi hi hello")

{'input_ids': [101, 7632, 7632, 7592, 102], 'attention_mask': [1, 1, 1, 1, 1]}

In [41]:
토크나이저("hi hi hello hello hello")

{'input_ids': [101, 7632, 7632, 7592, 7592, 7592, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [42]:
토크나이저(["hi hello"])

{'input_ids': [[101, 7632, 7592, 102]], 'attention_mask': [[1, 1, 1, 1]]}

`-` 기본사용2: 텍스트 혹은 텍스트의 리스트, 리스트의 리스트를 전달가능

In [43]:
토크나이저(["hi hello", "hello hello", "hi hi"])

{'input_ids': [[101, 7632, 7592, 102], [101, 7592, 7592, 102], [101, 7632, 7632, 102]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]}

`-` `truncation=True` 의 역할

In [46]:
dct = 토크나이저('hi hello'*1000,truncation=True)
dct

{'input_ids': [101, 7632, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 7592, 4048, 75

In [47]:
len(dct['input_ids']), len(dct['attention_mask'])

(512, 512)

`-` maxlen, padding, attention_mask

In [48]:
dct = 토크나이저('hi hello',max_length=10,padding="max_length")
dct

{'input_ids': [101, 7632, 7592, 102, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]}

# 5. 인공지능 ($\star$)

## A. 1단계

> 인공지능에 대한 이해

`-` 인공지능 불러오기

In [253]:
torch.random.manual_seed(43052)
인공지능 = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", 
    num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

`-` 인공지능의 정체? 엄청나게 많은 숫자들이 포함된 어떠한 물체 (엄청나게
많은 파라메터들이 포함된 네트워크)

In [254]:
인공지능

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [255]:
인공지능.classifier.weight

Parameter containing:
tensor([[-0.0234,  0.0279,  0.0242,  ...,  0.0091, -0.0063, -0.0133],
        [ 0.0087,  0.0007, -0.0099,  ...,  0.0183, -0.0007,  0.0295]],
       requires_grad=True)

In [256]:
인공지능.pre_classifier.weight

Parameter containing:
tensor([[-0.0122,  0.0030,  0.0301,  ...,  0.0003,  0.0327, -0.0123],
        [-0.0041,  0.0008,  0.0169,  ..., -0.0163, -0.0117, -0.0135],
        [ 0.0020, -0.0215, -0.0021,  ...,  0.0016,  0.0102, -0.0483],
        ...,
        [-0.0026, -0.0327,  0.0099,  ...,  0.0341, -0.0184, -0.0109],
        [ 0.0088, -0.0345, -0.0011,  ...,  0.0018,  0.0172, -0.0122],
        [-0.0148,  0.0147, -0.0184,  ..., -0.0166,  0.0241,  0.0201]],
       requires_grad=True)

`-` 인공지능? “입력정보 -\> 정리된숫자 -\> 계산 -\> 계산된숫자 -\>
출력정보” 의 과정에서 “계산”을 담당

-   인공지능이 가지고 있는 숫자들은 계산에 사용되는 숫자들임..

`-` 입력정보에 영화에 대한 부정적 평가를 넣는다면?

In [365]:
입력정보_원시텍스트 = "This movie was a huge disappointment."
정리된숫자_토큰화된자료 = 토크나이저(입력정보_원시텍스트,truncation=True,return_tensors='pt')
정리된숫자_토큰화된자료

{'input_ids': tensor([[  101,  2023,  3185,  2001,  1037,  4121, 10520,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [366]:
인공지능(**정리된숫자_토큰화된자료)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1173,  0.0261]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [368]:
계산된숫자_로짓 = 인공지능(**정리된숫자_토큰화된자료).logits.detach().numpy()
계산된숫자_로짓

array([[-0.11731046,  0.02610319]], dtype=float32)

In [370]:
출력정보_확률 = np.exp(계산된숫자_로짓) / np.exp(계산된숫자_로짓).sum()
출력정보_확률 # 0일확률, 1일확률

array([[0.46420792, 0.53579205]], dtype=float32)

In [371]:
출력정보_확률.argmax() # 부정적 영화평가에 대한 인공지능의 예측

1

In [372]:
계산된숫자_로짓.argmax() # 부정적 영화평가에 대한 인공지능의 예측 <-- 이렇게 구해도됩니다.. 왜??

1

`-` 입력정보에 영화에 대한 긍정적 평가를 넣는다면?

In [373]:
입력정보_원시텍스트 = "This was a masterpiece."
정리된숫자_토큰화된자료 = 토크나이저(입력정보_원시텍스트,truncation=True,return_tensors='pt')
정리된숫자_토큰화된자료

{'input_ids': tensor([[  101,  2023,  2001,  1037, 17743,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [374]:
인공지능(**정리된숫자_토큰화된자료)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1080,  0.0264]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [375]:
계산된숫자_로짓 = 인공지능(**정리된숫자_토큰화된자료).logits.detach().numpy()
계산된숫자_로짓

array([[-0.10802954,  0.02638793]], dtype=float32)

In [377]:
출력정보_확률 = np.exp(계산된숫자_로짓) / np.exp(계산된숫자_로짓).sum()
출력정보_확률 # 0일확률, 1일확률

array([[0.46644613, 0.53355384]], dtype=float32)

In [378]:
출력정보_확률.argmax()
계산된숫자_로짓.argmax() # 부정적 영화평가에 대한 인공지능의 예측

1

`-` 아무숫자나 뱉어내는듯 $\to$ 멍청한 인공지능.. (옹호: 당연하지
학습전이니까)

`-` 인공지능에 대한 이해1: 인공지능은 “정리된숫자”를 입력으로 하고
일련의 계산을 거쳐 “계산된숫자”를 출력해주는 함수라 생각할 수 있음.

`-` 인공지능에 대한 이해2: 인공지능은 (1) 많은숫자들과 (2) 고유의
계산방식을 가지고 있음.

-   인공지능이 내부에 자체적으로 저장하고 있는 숫자를 **파라메터**라고
    부름.
-   인공지능은 나름의 법칙에 따라 “데이터”와 “파라메터”를 계산함. 즉
    인공지능은 자체적으로 데이터와 파라메터를 어떻게 계산할지
    알고있는데, 이러한 고유의 계산방식을 **아키텍처**라고 말함.

`-` 인공지능에 대한 이해3: 두 인공지능이 서로 다른 고유의 계산방식을
가지고 있다면 두 인공지능은 “다른 모델”임.

`-` 인공지능에 대한 이해3’: 동일한 생성방식으로 만들어진 인공지능들은
모두 같은 모델임. 예를들면 아래의 인공지능1,2는 같은 모델임

``` python
인공지능1 = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", 
    num_labels=2
)
인공지능2 = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", 
    num_labels=2
)
```

`-` 인공지능에 대한 이해4: 두 인공지능이 같은 모델이라고 해도, 항상 같은
결과를 주는건 아님. 파라메터에 따라 다른결과를 줄 수 도 있음. (예를들면
위의 인공지능1,2는 같은 모델이지만 다른 파라메터를 가지므로 다른 결과를
줌)

## B. 2단계

> 미니배치를 이해하자.

`-` 예비학습

In [379]:
arr = np.array([[1,2],[2,3],[3,4]])
arr

array([[1, 2],
       [2, 3],
       [3, 4]])

In [380]:
arr / arr.sum(axis=1).reshape(-1,1)

array([[0.33333333, 0.66666667],
       [0.4       , 0.6       ],
       [0.42857143, 0.57142857]])

`-` 예비개념1: 인공지능은 사실 영화평을 하나씩 하나씩 처리하지 않는다.
덩어리로 처리한다.

`-` 예비개념2: 그렇다고 해서 인공지능이 25000개를 모두 덩어리로
처리하는건 아니다 $\to$ 16개씩, 혹은 32개씩 묶어서 작은덩어리로 만든 후
처리한다.

-   16,32 와 같은 숫자를 `batch_size` 라고 한다.
-   16개, 32개로 모인 작은덩어리를 미니배치라고 한다.

`-` 16개의 입력정보를 한번에 처리

In [381]:
입력정보들_원시텍스트 = 데이터['train'][:16]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시텍스트,truncation=True,return_tensors='pt',padding=True)
계산된숫자들_로짓 = 인공지능(**정리된숫자들_토큰화된자료).logits.data.numpy()
출력정보들_확률 = np.exp(계산된숫자들_로짓) / np.exp(계산된숫자들_로짓).sum(axis=1).reshape(-1,1)
출력정보들_확률

array([[0.47823825, 0.5217617 ],
       [0.47215328, 0.5278467 ],
       [0.4804948 , 0.5195052 ],
       [0.46609667, 0.5339033 ],
       [0.48814487, 0.5118551 ],
       [0.48004225, 0.5199578 ],
       [0.49288097, 0.50711906],
       [0.49470255, 0.5052974 ],
       [0.4941453 , 0.50585467],
       [0.49016201, 0.50983804],
       [0.495789  , 0.504211  ],
       [0.48260576, 0.51739424],
       [0.49386355, 0.5061364 ],
       [0.49013382, 0.5098662 ],
       [0.48240176, 0.5175982 ],
       [0.49301967, 0.5069803 ]], dtype=float32)

In [382]:
#출력정보들_확률.argmax(axis=1) 
계산된숫자들_로짓.argmax(axis=1)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

`-` 기억할 것: `정리된숫자들_토큰화된자료` 는 모두 길이가 512임. (그렇게
되도록 패딩함)

In [383]:
정리된숫자들_토큰화된자료['input_ids'].shape, 정리된숫자들_토큰화된자료['attention_mask'].shape

(torch.Size([16, 512]), torch.Size([16, 512]))

`-` 실제 단어수

In [384]:
정리된숫자들_토큰화된자료['attention_mask'].sum(axis=1)

tensor([363, 304, 133, 185, 495, 154, 143, 388, 512, 297, 365, 171, 192, 173,
        470, 263])

`-` 패딩된단어수

In [385]:
512 - 정리된숫자들_토큰화된자료['attention_mask'].sum(axis=1)

tensor([149, 208, 379, 327,  17, 358, 369, 124,   0, 215, 147, 341, 320, 339,
         42, 249])

`-` 이러한 변환이 이루어지는 이유? 인공지능은 항상 `(n,m)` 차원으로
정리된 정리된숫자들을 입력으로 받아야함.

-   왜? 사실 인공지능은 행렬계산을 하도록 설계되어있음.
-   그래서 할수없이 패딩처리를 해야하는 것임. (실제로는 행렬로 만들수
    없지만 억지로 만들기 위해서..)

## C. 3단계

> 동적패딩을 이해하자.

`-` 만약에 `batch_size=4` 로 설정하여 처리한다면?

In [386]:
입력정보들_원시텍스트 = 데이터['train'][:4]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시텍스트,truncation=True,return_tensors='pt',padding=True)
계산된숫자들_로짓 = 인공지능(**정리된숫자들_토큰화된자료).logits.data.numpy()
출력정보들_확률 = np.exp(계산된숫자들_로짓) / np.exp(계산된숫자들_로짓).sum(axis=1).reshape(-1,1)
출력정보들_확률

array([[0.47823825, 0.5217618 ],
       [0.47215328, 0.5278467 ],
       [0.4804948 , 0.5195052 ],
       [0.46609667, 0.5339033 ]], dtype=float32)

In [387]:
계산된숫자들_로짓.argmax(axis=1)

array([1, 1, 1, 1])

`-` `정리된숫자들_토큰화된자료`의 차원은 어떠할까? (4,512)로 예상되지
않을까?

In [388]:
정리된숫자들_토큰화된자료['input_ids'].shape, 정리된숫자들_토큰화된자료['attention_mask'].shape

(torch.Size([4, 363]), torch.Size([4, 363]))

-   끝차원이 512가 아니라 363이다.. 왜??

`-` 덩어리의 상태에 따라서 유동적으로 패딩 $\to$ 그래도 잘 돌아감

In [389]:
입력정보들_원시데이터 = 데이터['train'][:4]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시데이터,truncation=True,return_tensors='pt',padding=True)
print(정리된숫자들_토큰화된자료['input_ids'].shape)
print(인공지능(**정리된숫자들_토큰화된자료))

torch.Size([4, 363])
SequenceClassifierOutput(loss=None, logits=tensor([[-0.0847,  0.0024],
        [-0.0830,  0.0285],
        [-0.0600,  0.0180],
        [-0.0919,  0.0440]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [390]:
입력정보들_원시데이터 = 데이터['train'][4:8]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시데이터,truncation=True,return_tensors='pt',padding=True)
print(정리된숫자들_토큰화된자료['input_ids'].shape)
print(인공지능(**정리된숫자들_토큰화된자료))

torch.Size([4, 495])
SequenceClassifierOutput(loss=None, logits=tensor([[-0.0422,  0.0053],
        [-0.0646,  0.0152],
        [-0.0172,  0.0112],
        [-0.0283, -0.0072]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [391]:
입력정보들_원시데이터 = 데이터['train'][8:12]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시데이터,truncation=True,return_tensors='pt',padding=True)
print(정리된숫자들_토큰화된자료['input_ids'].shape)
print(인공지능(**정리된숫자들_토큰화된자료))

torch.Size([4, 512])
SequenceClassifierOutput(loss=None, logits=tensor([[-0.0246, -0.0012],
        [-0.0594, -0.0200],
        [-0.0240, -0.0071],
        [-0.0836, -0.0140]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [392]:
입력정보들_원시데이터 = 데이터['train'][12:16]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시데이터,truncation=True,return_tensors='pt',padding=True)
print(정리된숫자들_토큰화된자료['input_ids'].shape)
print(인공지능(**정리된숫자들_토큰화된자료))

torch.Size([4, 470])
SequenceClassifierOutput(loss=None, logits=tensor([[-0.0440, -0.0195],
        [-0.0469, -0.0074],
        [-0.0542,  0.0162],
        [-0.0316, -0.0037]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`-` 싹다 512로 통일한것대비 큰 차이없음..

In [393]:
입력정보들_원시데이터 = 데이터['train'][:16]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시데이터,truncation=True,return_tensors='pt',padding=True)
print(정리된숫자들_토큰화된자료['input_ids'].shape)
print(인공지능(**정리된숫자들_토큰화된자료))

torch.Size([16, 512])
SequenceClassifierOutput(loss=None, logits=tensor([[-0.0847,  0.0024],
        [-0.0830,  0.0285],
        [-0.0600,  0.0180],
        [-0.0919,  0.0440],
        [-0.0422,  0.0053],
        [-0.0646,  0.0152],
        [-0.0172,  0.0112],
        [-0.0283, -0.0072],
        [-0.0246, -0.0012],
        [-0.0594, -0.0200],
        [-0.0240, -0.0071],
        [-0.0836, -0.0140],
        [-0.0440, -0.0195],
        [-0.0469, -0.0074],
        [-0.0542,  0.0162],
        [-0.0316, -0.0037]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## D. 4단계

> 손실(=loss)의 개념을 이해하자.

`-` `정리된숫자들_토큰화된자료`에서 `labels` 를 추가 전달하면
`인공지능(**정리된숫자들_토큰화된자료)`의 결과로 `loss`가 추가계산됨

In [394]:
입력정보들_원시데이터 = 데이터['train'][:4]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시데이터,truncation=True,return_tensors='pt',padding=True)
#데이터['train'][:4]['label']
정리된숫자들_토큰화된자료['labels'] = torch.tensor([0,0,0,0]) # 정답입력
인공지능(**정리된숫자들_토큰화된자료)

SequenceClassifierOutput(loss=tensor(0.7461, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0847,  0.0024],
        [-0.0830,  0.0285],
        [-0.0600,  0.0180],
        [-0.0919,  0.0440]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`-` 정리를 해보자.

In [412]:
입력정보들_원시데이터 = 데이터['train'][:4]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시데이터,truncation=True,return_tensors='pt',padding=True)
print(f'실제정답: {데이터['train'][:4]['label']}')
정리된숫자들_토큰화된자료['labels'] = torch.tensor(데이터['train'][:4]['label']) # 정답입력
계산된숫자들_로짓 = 인공지능(**정리된숫자들_토큰화된자료).logits.detach().numpy()
출력정보들_확률 = np.exp(계산된숫자들_로짓) / np.exp(계산된숫자들_로짓).sum(axis=1).reshape(-1,1)
print(f'인공지능의예측: {출력정보들_확률.argmax(axis=1)}')
print(f'인공지능확신정도: {출력정보들_확률.max(axis=1)}')
print(f'손실: {인공지능(**정리된숫자들_토큰화된자료).loss.item():.4f}')

실제정답: [0, 0, 0, 0]
인공지능의예측: [1 1 1 1]
인공지능확신정도: [0.5217618 0.5278467 0.5195052 0.5339033]
손실: 0.7461

`-` loss는 작을수록 좋은 것임. 문제를 많이 틀릴수록 loss가 큼, 문제를
조금 틀릴수록 loss가 작음

**텍스트0~텍스트3**

In [413]:
입력정보들_원시데이터 = 데이터['train'][:4]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시데이터,truncation=True,return_tensors='pt',padding=True)
print(f'실제정답: {데이터['train'][:4]['label']}')
정리된숫자들_토큰화된자료['labels'] = torch.tensor(데이터['train'][:4]['label']) # 정답입력
계산된숫자들_로짓 = 인공지능(**정리된숫자들_토큰화된자료).logits.detach().numpy()
출력정보들_확률 = np.exp(계산된숫자들_로짓) / np.exp(계산된숫자들_로짓).sum(axis=1).reshape(-1,1)
print(f'인공지능의예측: {출력정보들_확률.argmax(axis=1)}')
print(f'인공지능확신정도: {출력정보들_확률.max(axis=1)}')
print(f'손실: {인공지능(**정리된숫자들_토큰화된자료).loss.item():.4f}')

실제정답: [0, 0, 0, 0]
인공지능의예측: [1 1 1 1]
인공지능확신정도: [0.5217618 0.5278467 0.5195052 0.5339033]
손실: 0.7461

**텍스트12498~텍스트12501**

In [414]:
입력정보들_원시데이터 = 데이터['train'][12498:12502]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시데이터,truncation=True,return_tensors='pt',padding=True)
print(f'실제정답: {데이터['train'][12498:12502]['label']}')
정리된숫자들_토큰화된자료['labels'] = torch.tensor(데이터['train'][12498:12502]['label']) # 정답입력
계산된숫자들_로짓 = 인공지능(**정리된숫자들_토큰화된자료).logits.detach().numpy()
출력정보들_확률 = np.exp(계산된숫자들_로짓) / np.exp(계산된숫자들_로짓).sum(axis=1).reshape(-1,1)
print(f'인공지능의예측: {출력정보들_확률.argmax(axis=1)}')
print(f'인공지능확신정도: {출력정보들_확률.max(axis=1)}')
print(f'손실: {인공지능(**정리된숫자들_토큰화된자료).loss.item():.4f}')

실제정답: [0, 0, 1, 1]
인공지능의예측: [1 1 1 1]
인공지능확신정도: [0.52731967 0.5243837  0.51578873 0.5351162 ]
손실: 0.6950

`-` 똑같이 틀려도 오답에 대한 확신이 강할수록 loss가 크다.

**텍스트0~텍스트1**

In [415]:
입력정보들_원시데이터 = 데이터['train'][:2]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시데이터,truncation=True,return_tensors='pt',padding=True)
print(f'실제정답: {데이터['train'][:2]['label']}')
정리된숫자들_토큰화된자료['labels'] = torch.tensor(데이터['train'][:2]['label']) # 정답입력
계산된숫자들_로짓 = 인공지능(**정리된숫자들_토큰화된자료).logits.detach().numpy()
출력정보들_확률 = np.exp(계산된숫자들_로짓) / np.exp(계산된숫자들_로짓).sum(axis=1).reshape(-1,1)
print(f'인공지능의예측: {출력정보들_확률.argmax(axis=1)}')
print(f'인공지능확신정도: {출력정보들_확률.max(axis=1)}')
print(f'손실: {인공지능(**정리된숫자들_토큰화된자료).loss.item():.4f}')

실제정답: [0, 0]
인공지능의예측: [1 1]
인공지능확신정도: [0.5217617 0.5278467]
손실: 0.7440

**텍스트1~텍스트2**

In [416]:
입력정보들_원시데이터 = 데이터['train'][1:3]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시데이터,truncation=True,return_tensors='pt',padding=True)
print(f'실제정답: {데이터['train'][1:3]['label']}')
정리된숫자들_토큰화된자료['labels'] = torch.tensor(데이터['train'][1:3]['label']) # 정답입력
계산된숫자들_로짓 = 인공지능(**정리된숫자들_토큰화된자료).logits.detach().numpy()
출력정보들_확률 = np.exp(계산된숫자들_로짓) / np.exp(계산된숫자들_로짓).sum(axis=1).reshape(-1,1)
print(f'인공지능의예측: {출력정보들_확률.argmax(axis=1)}')
print(f'인공지능확신정도: {출력정보들_확률.max(axis=1)}')
print(f'손실: {인공지능(**정리된숫자들_토큰화된자료).loss.item():.4f}')

실제정답: [0, 0]
인공지능의예측: [1 1]
인공지능확신정도: [0.5278467 0.5195052]
손실: 0.7417

`-` 손실 = 인공지능의 “멍청한정도” 혹은 “똑똑한정도”을 숫자화한것

> **$\star\star\star$ 학습이 가능한 이유 (대충 아이디어만)**
>
> `1`. 랜덤으로 1개의 인공지능을 생성한다. (아래의 코드로 가능)
>
> ``` python
> 인공지능 = 인공지능생성기()
> ```
>
> `2`. 인공지능의 파라메터중 하나를 찍는다. 예를들면 아래와 같은 상황이
> 있다고 하자.
>
> ``` python
> 인공지능.classifier.weight
> ```
>
>     Parameter containing:
>     tensor([[-0.0234,  0.0279,  0.0242,  ...,  0.0091, -0.0063, -0.0133],
>             [ 0.0087,  0.0007, -0.0099,  ...,  0.0183, -0.0007,  0.0295]],
>            requires_grad=True)
>
> 하나의 숫자 `-0.0234` 를 선택한다.
>
> `3`. `-0.0234`의 값을 조금 변화시켜본다. 예를들면, `-0.0235`,
> `-0.0233` 와 같은 식으로 변화시켜본뒤 loss를 관찰한다.
>
> `4`. 원래값과 변화시킨값들중 loss를 가장 작게 만드는 값으로
> `-0.0234`를 update한다.
>
> `5`. 다른 모든 파라메터에 대하여 1~4를 반복한다. (과정을 반복할수록
> loss가 작아지겠죠, 즉 인공지능은 똑똑해지겠죠)

`-` 인공지능의 학습은 마법같은 신비한 현상이 아니고 극한의 노가다를 통해
얻어지는 산물일 뿐이다.

# 6. 데이터전처리2

In [401]:
def 데이터전처리하기2(examples):
    return 데이터전처리하기1(examples["text"], truncation=True)

In [402]:
데이터 = datasets.load_dataset('imdb')
전처리된데이터 = 데이터.map(데이터전처리하기2,batched=True)

Map: 100%|█████████████████████████████████████████████████████████████████████████| 25000/25000 [00:03<00:00, 8260.93 examples/s]

# 7. 데이터콜렉터

`-` 데이터콜렉터 사용방법

In [407]:
데이터콜렉터 = 동적패딩처리하기 = transformers.DataCollatorWithPadding(tokenizer=토크나이저,return_tensors='pt')
데이터콜렉터

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
    0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
    100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
    101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
    102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
    103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None

In [408]:
작은덩어리_딕셔너리 = 전처리된데이터['train'][:4]
작은덩어리_딕셔너리.keys()

dict_keys(['text', 'label', 'input_ids', 'attention_mask'])

In [409]:
작은덩어리_리스트 = [{
    'label': 작은덩어리_딕셔너리['label'][i],
    'input_ids': 작은덩어리_딕셔너리['input_ids'][i],
    'attention_mask': 작은덩어리_딕셔너리['attention_mask'][i]
} for i in range(4)]

In [410]:
데이터콜렉터(작은덩어리_리스트)

{'input_ids': tensor([[  101,  1045, 12524,  ...,  5436,  1012,   102],
        [  101,  1000,  1045,  ...,     0,     0,     0],
        [  101,  2065,  2069,  ...,     0,     0,     0],
        [  101,  2023,  2143,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([0, 0, 0, 0])}

In [411]:
인공지능(**데이터콜렉터(작은덩어리_리스트))

SequenceClassifierOutput(loss=tensor(0.7461, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0847,  0.0024],
        [-0.0830,  0.0285],
        [-0.0600,  0.0180],
        [-0.0919,  0.0440]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`-` 데이터콜렉터의 기능: batch_size 에 따라서 패딩시켜줌

# 8. 평가하기

`-` `accuracy.compute`의 기능

In [348]:
accuracy = evaluate.load("accuracy")

In [356]:
accuracy.compute(references=[0, 0, 0], predictions=[0, 1, 1])

{'accuracy': 0.3333333333333333}

`-` 함수내용

In [358]:
# def 평가하기(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     accuracy = evaluate.load("accuracy")
#     return accuracy.compute(predictions=predictions, references=labels)

In [417]:
def 평가하기(eval_pred):
    계산된숫자들_로짓, 실제정답 = eval_pred
    인공지능의예측 = np.argmax(계산된숫자들_로짓, axis=1)
    accuracy = evaluate.load("accuracy")
    return accuracy.compute(predictions=인공지능의예측, references=실제정답)

> `평가하기` 함수는 `eval_dataset`에 적용됨

# 9. 트레이너 세부지침

In [418]:
트레이너세부지침 = transformers.TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2, # 전체문제세트를 2번 공부하라..
    weight_decay=0.01,
    eval_strategy="epoch", 
    save_strategy="epoch", 
    load_best_model_at_end=True,
    push_to_hub=False,
)

> **옵션 설명:**
>
> 1.  **`output_dir="my_awesome_model"`**:
>     -   학습된 모델과 관련 파일(예: 체크포인트, 로그 등)이 저장될
>         디렉토리 경로를 지정합니다.
> 2.  **`learning_rate=2e-5`**:
>     -   학습률(learning rate)을 설정합니다. 모델이 가중치를 업데이트할
>         때 사용하는 스텝 크기로, 작은 값일수록 학습 속도가 느려지지만
>         안정성이 높습니다.
> 3.  **`per_device_train_batch_size=16`**:
>     -   **훈련** 시, 각 GPU 또는 CPU 디바이스에서 사용할 배치 크기를
>         설정합니다. 이 경우 각 디바이스에서 16개의 샘플을 한 번에
>         처리합니다.
> 4.  **`per_device_eval_batch_size=16`**:
>     -   **평가** 시, 각 GPU 또는 CPU 디바이스에서 사용할 배치 크기를
>         설정합니다. 평가 시에는 훈련과 같은 배치 크기를 사용하는 것이
>         일반적입니다.
> 5.  **`num_train_epochs=2`**:
>     -   전체 훈련 데이터셋을 몇 번 반복(에포크)할지 설정합니다.
>         여기서는 데이터셋 전체를 2번 학습하게 됩니다.
> 6.  **`weight_decay=0.01`**:
>     -   가중치 감쇠(weight decay)를 적용하여 모델의 가중치가 지나치게
>         커지지 않도록 제어합니다. 0.01의 값은 가중치가 조금씩 감소하게
>         하여 모델의 일반화 성능을 높이는 데 도움을 줄 수 있습니다.
> 7.  **`eval_strategy="epoch"`**:
>     -   평가 전략을 설정합니다. 여기서는 `epoch`으로 설정되어, 매
>         에포크가 끝날 때마다 평가가 진행됩니다.
>     -   다른 값으로는 `steps` (일정한 스텝마다 평가) 등이 있습니다.
> 8.  **`save_strategy="epoch"`**:
>     -   모델을 언제 저장할지 설정합니다. `epoch`으로 설정되면, 매
>         에포크가 끝날 때마다 체크포인트를 저장합니다.
>     -   다른 값으로는 `steps` (일정한 스텝마다 저장) 등이 있습니다.
> 9.  **`load_best_model_at_end=True`**:
>     -   학습이 끝난 후, 가장 성능이 좋았던 체크포인트를 불러옵니다.
>         평가 지표에 따라 가장 성능이 좋았던 모델을 자동으로 불러와
>         최종 모델로 사용하게 됩니다.
> 10. **`push_to_hub=False`**:
>     -   모델 학습이 끝난 후 Hugging Face Hub에 모델을 업로드할지
>         여부를 설정합니다. `False`로 설정하면 업로드하지 않습니다.
>     -   `True`로 설정하면 모델과 관련된 파일들이 Hugging Face Hub에
>         업로드되어 다른 사람들과 공유할 수 있습니다.

`-` lr = 학습률 = 답이 틀렸을 경우 혼내는 정도

-   정확한 느낌은 경사하강법을 이해해야함.

# 10. 트레이너

In [ ]:
트레이너 = transformers.Trainer(
    model=인공지능,
    args=트레이너세부지침,
    train_dataset=전처리된데이터['train'],
    eval_dataset=전처리된데이터['test'],
    #tokenizer=토크나이저, # 왜 필요하지??
    data_collator=데이터콜렉터,
    compute_metrics=평가하기,
)
트레이너.train()

# 11. 파이프라인

`-` 강인공지능?

> ref: <https://zdnet.co.kr/view/?no=20160622145838>

In [ ]:
강인공지능 = transformers.pipeline("sentiment-analysis", model="my_awesome_model/checkpoint-1563")
print(강인공지능("This movie was a huge disappointment."))
print(강인공지능("This was a masterpiece."))